# Importing libraries

In [1]:
import os
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential # class of NN
from keras.layers import Convolution2D # class for adding convulution
from keras.layers import MaxPooling2D # pooling layer class
from keras.layers import Flatten # flatten layer class
from keras.layers import Dense, Dropout # other normal layers in NN
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from joblib import dump

In [3]:
os.listdir('datasets')

['test_set', 'training_set']

### Initialising the CNN

In [6]:
classifier = Sequential()

classifier.add(Convolution2D(filters = 16,
                            kernel_size = (5, 5),
                            input_shape = (512, 512, 1),
                            activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))
classifier.add(Convolution2D(filters = 32,
                            kernel_size = (5, 5),
                            activation = 'relu'))

### Adding a second maxpooling layer

In [7]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))

### Flattening

In [8]:
classifier.add(Flatten())

### Full connection

In [9]:
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

### Compiling the CNN

In [10]:
classifier.compile(optimizer = 'adam',
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])

In [11]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 508, 508, 16)      416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 254, 254, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 254, 254, 16)      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 250, 250, 32)      12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 125, 125, 32)     0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 125, 125, 32)     

In [12]:
def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

In [13]:
train_datagen = ImageDataGenerator(rescale = 1/255,
                                    preprocessing_function = to_grayscale_then_rgb)

test_datagen = ImageDataGenerator(rescale = 1/255, preprocessing_function = to_grayscale_then_rgb)

In [14]:
tr = train_datagen.flow_from_directory(
        'datasets/training_set',
        target_size = (512, 512),
        batch_size = 2,
        class_mode = 'binary',
        color_mode = 'grayscale')

tx = test_datagen.flow_from_directory(
        'datasets/test_set',
        target_size = (512, 512),
        batch_size = 2,
        class_mode = 'binary',
        color_mode = 'grayscale')

Found 200 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [15]:
tr.image_shape

(512, 512, 1)

In [16]:
tx.image_shape

(512, 512, 1)

In [20]:
history = classifier.fit(tr,
                        steps_per_epoch = 5,
                        epochs = 50,
                        validation_data = tx,
                        validation_steps = 20)

InvalidArgumentError: Matrix size-incompatible: In[0]: [262144,1], In[1]: [3,1] [Op:MatMul]